In [ ]:
from metaflow import FlowSpec, step
import sys
import os
from pathlib import Path

path = Path(os.getcwd())
sys.path.append(str(path.parent))

import nlopt_z_match
import numpy as np
import math
import pandas as pd
import imp
import time
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import logging_mod
import os

class ForeachFlow(FlowSpec):

    @step
    def start(self):
        self.filename = '../Animal_Data_study_backup.csv'
        weights = [1]*72
        weights[15] = 9
        weights[2] = 7
        weights[5] = 5
        weights[34] = 8
        weights[10] = 10
        weights[25] = 4.5
        weights[35] = 4.5
        weights[45] = 17.78
        weights[55] = 2.0
        self.weights_array = weights
        self.grad_calculation = ['finite','svd']
        self.imds_init_flag = 0
        self.next(self.a, foreach='grad_calculation')

    @step
    def a(self):
        print("Starting Backward MDS --- case 1 ",self.input)
        self.title = '%s processed' % self.input
        self.nlopt_weights, self.orig, self.nlopt_residual, self.fn_count = nlopt_z_match.main(self.filename,
                                                                          self.weights_array,
                                                                          self.input)
        print('The result is: %s' % self.nlopt_residual, self.fn_count)
        self.next(self.newstep)

    @step
    def newstep(self, inputs):
        self.results = [input.title for input in inputs]
        self.next(self.end)

    @step
    def end(self):
        print('\n'.join(self.results))

if __name__ == '__main__':
    ForeachFlow()


In [ ]:
# To run the flow
python runner.py run

# To get a workflow graph
python runner.py output-dot | dot -Tpng -o graph.png

# To run a task